**Data collection**

  In this we colect the data from the dataset we obtained from the kagglefor Chatbot.

In [ ]:
data_file = '/content/dialogs.txt'  # Replace with your file path

conversations = []

with open(data_file, 'r', encoding='utf-8') as file:
    for line in file:
        conversations.append(line.strip())

# Now, the 'conversations' list contains your data.


**Data Preprocessing**

In [ ]:
import nltk
import string
import re

# Download necessary NLTK resources (if you haven't already)
nltk.download('punkt')

from nltk.tokenize import word_tokenize

# Define a function to preprocess and format text
def preprocess_text(text):
    # Tokenization: Split the text into words
    tokens = word_tokenize(text)

    # Cleaning: Remove punctuation, convert to lowercase, and remove extra whitespace
    cleaned_tokens = [token.lower() for token in tokens if token not in string.punctuation]
    cleaned_text = ' '.join(cleaned_tokens)

    return cleaned_text

# Read the text dataset from a file
data_file = '/content/dialogs.txt'  # Replace with your dataset file path

with open(data_file, 'r', encoding='utf-8') as file:
    dataset = [line.strip() for line in file]

# Preprocess and format the entire dataset
preprocessed_dataset = [preprocess_text(sentence) for sentence in dataset]

# Optionally, you can save the preprocessed dataset to a new file for later use
output_file = 'preprocessed_dataset.txt'

with open(output_file, 'w', encoding='utf-8') as file:
    for sentence in preprocessed_dataset:
        file.write(sentence + '\n')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load a pre-trained model
model_name = "gpt2-large"  # Change this to "gpt3" or "gpt4" for different models
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Now, you have the pre-trained model and tokenizer ready to use.


In [ ]:
import torch
from transformers import TrainingArguments
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2LMHeadModel, GPT2Tokenizer, DataCollatorForLanguageModeling, Trainer, TrainingArguments

# Load pre-trained GPT-2 model and tokenizer
model_name = "gpt2"  # Change to the desired model name (e.g., "gpt2-medium", "gpt2-large")
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Read the conversation dataset from a text file
data_file = "preprocessed_dataset.txt"

with open(data_file, "r", encoding="utf-8") as file:
    dataset = file.read().splitlines()

# Tokenize the dataset and prepare it for training
input_ids = []
attention_mask = []

for example in dataset:
    encoded = tokenizer.encode(example, return_tensors="pt", max_length=1024, truncation=True)
    input_ids.append(encoded)
    attention_mask.append(torch.ones_like(encoded))  # Assuming all tokens are relevant

# Create a custom PyTorch dataset
class CustomDataset(Dataset):
    def __init__(self, input_ids, attention_mask):
        self.input_ids = input_ids
        self.attention_mask = attention_mask

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            "input_ids": self.input_ids[idx],
            "attention_mask": self.attention_mask[idx]
        }

custom_dataset = CustomDataset(input_ids, attention_mask)

# Set up training arguments
training_args = TrainingArguments(
    output_dir="./gpt2-large",  # Change to your desired output directory
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=8,
    save_steps=1000,
    save_total_limit=2,
)

# Create data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

# Initialize Trainer and start training
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=custom_dataset,
)

# Start training
trainer.train()

# Save the fine-tuned model
model.save_pretrained("./gpt2-large")

# You can now use this fine-tuned model for text generation or chatbot tasks.


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer

# Define hyperparameters
learning_rates = [1e-5, 3e-5, 1e-4]
batch_sizes = [4, 8, 16]
model_architectures = ["gpt2", "gpt2-medium", "gpt2-large"]

# Define a function to train and evaluate the chatbot with different hyperparameters
def train_and_evaluate(learning_rate, batch_size, model_architecture):
    # Load pre-trained model and tokenizer
    model = AutoModelForCausalLM.from_pretrained(model_architecture)
    tokenizer = AutoTokenizer.from_pretrained(model_architecture)

    # Set up training arguments
    training_args = TrainingArguments(
        output_dir=f"./chatbot_model_{model_architecture}_{learning_rate}_{batch_size}",
        overwrite_output_dir=True,
        num_train_epochs=3,
        per_device_train_batch_size=batch_size,
        save_steps=1000,
        save_total_limit=2,
        learning_rate=learning_rate,
    )

    # Create a Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=preprocessed_dataset.txt,  # Replace with your dataset
    )

    # Start training
    trainer.train()

    # Evaluate the model (you can use a validation set)
    # evaluation_results = trainer.evaluate()

    # Return any relevant evaluation results or metrics

# Hyperparameter tuning loop
for learning_rate in learning_rates:
    for batch_size in batch_sizes:
        for model_architecture in model_architectures:
            print(f"Training with learning rate: {learning_rate}, batch size: {batch_size}, model: {model_architecture}")
            train_and_evaluate(learning_rate, batch_size, model_architecture)


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer

# Define hyperparameters to tune
learning_rates = [1e-5, 3e-5, 5e-5]
batch_sizes = [4, 8, 16]
model_architectures = ["gpt2", "gpt2-medium", "gpt2-large"]

# Define a function to train and evaluate the chatbot with different hyperparameters
def train_and_evaluate(learning_rate, batch_size, model_name):
    # Load pre-trained model and tokenizer
    model = AutoModelForCausalLM.from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # Set up training arguments
    training_args = TrainingArguments(
        output_dir=f"./chatbot_model_{model_name}_{learning_rate}_{batch_size}",
        overwrite_output_dir=True,
        num_train_epochs=3,
        per_device_train_batch_size=batch_size,
        save_steps=1000,
        save_total_limit=2,
        learning_rate=learning_rate,
    )

    # Create a Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dialogs.txt,  # Replace with your training dataset
        eval_dataset=preprocessed_dataset.txt,    # Replace with your evaluation dataset
    )

    # Start training
    trainer.train()

    # Evaluate the model (you can use a validation set)
    evaluation_results = trainer.evaluate()

    return evaluation_results

# Hyperparameter tuning loop
best_hyperparameters = None
best_score = float('-inf')

for learning_rate in learning_rates:
    for batch_size in batch_sizes:
        for model_architecture in model_architectures:
            print(f"Training with learning rate: {learning_rate}, batch size: {batch_size}, model: {model_architecture}")
            evaluation_results = train_and_evaluate(learning_rate, batch_size, model_architecture)

            if evaluation_results["eval_loss"] > best_score:
                best_score = evaluation_results["eval_loss"]
                best_hyperparameters = (learning_rate, batch_size, model_architecture)

print(f"Best Hyperparameters: Learning Rate = {best_hyperparameters[0]}, Batch Size = {best_hyperparameters[1]}, Model = {best_hyperparameters[2]}")


In [ ]:
from flask import Flask, request, render_template
from transformers import AutoModelForCausalLM, AutoTokenizer

app = Flask(__name)

# Load the pre-trained chatbot model and tokenizer
model_name = "gpt2-large"  # Replace with your model name
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Define a function to generate responses
def generate_response(input_text, max_length=100):
    input_ids = tokenizer.encode(input_text, return_tensors="pt")
    response = model.generate(input_ids, max_length=max_length, num_return_sequences=1, no_repeat_ngram_size=2)
    return tokenizer.decode(response[0], skip_special_tokens=True)

@app.route("/", methods=["GET", "POST"])
def chatbot():
    if request.method == "POST":
        user_input = request.form["user_input"]
        bot_response = generate_response(user_input)
        return render_template("index.html", user_input=user_input, bot_response=bot_response)
    return render_template("index.html")

if __name__ == "__main__":
    app.run(debug=True)
